# CREATE FACT TABLE

In [0]:
df_silver = spark.sql("SELECT * FROM PARQUET.``abfss://silver@carstorageaccount.dfs.core.windows.net/carsales")

In [0]:
display(df_silver)

##### Reading all dims

In [0]:
df_dealer = spark.sql('''SELECT * FROM car_catalog.gold.dim_dealer''')
df_date = spark.sql('''SELECT * FROM car_catalog.gold.dim_date''')
df_branch = spark.sql('''SELECT * FROM car_catalog.gold.dim_branch''')
df_model = spark.sql('''SELECT * FROM car_catalog.gold.dim_model''')

###### Bringing keys to fact table

In [0]:
df_fact = df_silver.join(df_branch,df.silver['Branch_ID'] == df_branch['Branch_ID'],'left')\.
join(df_dealer,df.silver['Dealer_ID'] == df_dealer['Dealer_ID'],'left')\.
join(df_date,df.silver['Date'] == df_date['Date'],'left')\.
join(df_model,df.silver['Model_ID'] == df_model['Model_ID'],'left')
display(df_fact)

## Writing Fact Table

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists('factsales'):
    deltatbl = DeltaTable.forName(spark,'car_catalog.factsales')
    deltatbl.alias('t').merge(df_fact.alias('s'),'t.Date = s.Date AND t.Dealer_ID = s.Dealer_ID AND t.Model_ID = s.Model_ID AND t.Branch_ID = s.Branch_ID')\.
    whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
else:
    df_fact.write.format('delta')\
        .mode('overwrite')\
        .option("path","abfss://gold@carstorageaccount.dfs.core.windows.net/factsales")
        .saveAsTable('car_catalog.factsales')